In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [7]:
results = joblib.load('static_combined_BPW_frequency_feats_results.pkl')
data_types = ['Wake','N1', 'N2', 'N3', 'REM']

In [105]:
results.keys()

dict_keys([1, 2, 3, 4])

In [106]:
results[1].keys()

dict_keys(['Wake', 'N1', 'N2', 'N3', 'REM'])

In [107]:
results[1]['Wake'].keys()

dict_keys(['regional', 'regions_combined', 'all_feats'])

In [109]:
results[1]['Wake']['regional']

,random_state,y_true,Prefrontal_RF_y_preds,Prefrontal_DT_y_preds,Prefrontal_Ada_B_y_preds,Frontal_RF_y_preds,Frontal_DT_y_preds,Frontal_Ada_B_y_preds,Left Frontal_RF_y_preds,Left Frontal_DT_y_preds,Left Frontal_Ada_B_y_preds,Right Frontal_RF_y_preds,Right Frontal_DT_y_preds,Right Frontal_Ada_B_y_preds,Central_RF_y_preds,Central_DT_y_preds,Central_Ada_B_y_preds,Left Central_RF_y_preds,Left Central_DT_y_preds,Left Central_Ada_B_y_preds,Right Central_RF_y_preds,Right Central_DT_y_preds,Right Central_Ada_B_y_preds,Left Temporal_RF_y_preds,Left Temporal_DT_y_preds,Left Temporal_Ada_B_y_preds,Right Temporal_RF_y_preds,Right Temporal_DT_y_preds,Right Temporal_Ada_B_y_preds,Parietal_RF_y_preds,Parietal_DT_y_preds,Parietal_Ada_B_y_preds,Left Parietal_RF_y_preds,Left Parietal_DT_y_preds,Left Parietal_Ada_B_y_preds,Right Parietal_RF_y_preds,Right Parietal_DT_y_preds,Right Parietal_Ada_B_y_preds,Occipital_RF_y_preds,Occipital_DT_y_preds,Occipital_Ada_B_y_preds
0,1,"[[0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1], [0, 0...","[[0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 1, 0], [0, 0...","[[0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0], [1, 0...","[[0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0], [1, 0...","[[0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 1...","[[0, 1, 0, 1, 1, 0], [1, 1, 1, 0, 0, 0], [0, 1...","[[0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 1...","[[0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [1, 0...","[[0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [1, 0...","[[0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0], [0, 0...","[[1, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1], [1, 0...","[[1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 0], [1, 0...","[[0, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1...","[[0, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[0, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1...","[[0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0...","[[0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0...","[[1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1...","[[1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1], [0, 0...","[[0, 1, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0], [0, 0...","[[0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1], [0, 0...","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 1], [0, 1...","[[1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1], [0, 0...","[[1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 1, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0], [0, 1...","[[0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0...","[[0, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 1], [0, 0...","[[0, 1, 1, 1, 1, 0], [1, 0, 0, 0, 1, 1], [0, 0...","[[1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1...","[[1, 1, 0, 1, 1, 0], [0, 0, 1, 1, 0, 0], [0, 0...","[[0, 1, 0, 1, 1, 0], [1, 0, 0, 1, 0, 0], [0, 0...","[[0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0...","[[1, 0, 0, 1, 1, 0], [1, 0, 0, 0, 1, 0], [0, 1...","[[1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1], [0, 1..."
1,2,"[[0, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1], [0, 0...","[[0, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0], [1, 0...","[[0, 1, 0, 1, 1, 0], [0, 1, 0, 0, 0, 0], [1, 1...","[[0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0], [1, 1...","[[0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0], [1, 1...","[[0, 1, 0, 1, 1, 0], [0, 1, 0, 0, 0, 0], [1, 1...","[[0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1...","[[0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 1], [1, 1...","[[0, 0, 1, 1, 1, 0], [1, 0, 1, 0, 0, 1], [1, 0...","[[0, 0, 1, 1, 1, 0], [1, 0, 0, 0, 0, 1], [1, 1...","[[0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0], [1, 1...","[[1, 0, 0, 0, 1, 1], [0, 0, 0, 0, 1, 0], [1, 1...","[[1, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 0], [1, 0...","[[0, 1, 1, 1, 1, 0], [0, 0, 1, 0, 0, 0], [1, 1...","[[0, 1, 1, 1, 1, 0], [0, 0, 1, 0, 0, 0], [1, 1...","[[0, 1, 0, 1, 1, 0], [0, 1, 1, 0, 0, 0], [1, 1...","[[0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0...","[[1, 1, 0, 1, 0, 0], [1, 0, 1, 0, 1, 1], [1, 1...","[[0, 1, 0, 0

In [103]:
results_summaries = {}

expt_num = 1 


t1 = time.time()

for expt_num in [1,2,3,4]:
    
    features =  'regions_combined'
    features = 'all_feats'
    features = 'regional'
    loop_counter = 0
    for features in ['regional' , 'regions_combined' , 'all_feats']:
        # print('\n' + features + '\n')
        for data_type in data_types:
            # print('\n' + data_type + '\n')
            results_df = results[expt_num][data_type][features]
            
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
            
            sub_df_1 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('RF_', ''))
            sub_df_2 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('DT_', ''))
            sub_df_3 = pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T.rename(columns=lambda x: x.replace('Ada_B_', ''))
            
            df = pd.concat([sub_df_1, sub_df_2, sub_df_3])
            df.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
            df.insert(1, 'Max_acc',  df.iloc[:,1:].max(axis = 1).values)
    
            #Uncomment these two lines to display all of the results
            # display(df)
            mapping_dict = {'regional' : 'regional' , 'regions_combined' : 'regions_c' , 'all_feats' : 'all_feat' }
            if loop_counter == 0:
                df_out = df.iloc[:,:2].rename(columns = {'Max_acc' : data_type + '_' + mapping_dict[features] })
            else: 
                df_maxes = df.iloc[:,:2].rename(columns = {'Max_acc' : data_type + '_' + mapping_dict[features] })
                df_out = pd.concat([df_out , df_maxes], axis = 1)
    
            loop_counter +=1
                
                
    #Modify df_out to look how I want it to
    df_out = df_out.drop(columns = ['clf'] )
    df_out.insert(0, 'clf',  ['RF', 'DT', 'Ada_B'])
    df_out = df_out.set_index('clf')
    
    # display(df_out)
    
    df_rounded = (df_out*100).round(1)
    
    results_summaries[expt_num] = df_rounded

t2 = time.time()
t2-t1

17.820034503936768

In [ ]:
results_summaries[expt_num].

In [102]:
with pd.ExcelWriter("output.xlsx", engine="openpyxl") as writer:
    results_summaries[1].to_excel(writer, sheet_name="Sheet_1", startrow=2) # Default position: cell A1.
    results_summaries[2].to_excel(writer, sheet_name="Sheet_1", startrow=8, header=True) 
    results_summaries[3].to_excel(writer, sheet_name="Sheet_1", startrow=14, header=True) 
    results_summaries[4].to_excel(writer, sheet_name="Sheet_1", startrow=20, header=True) 

    #Access the ExcelWriter's ExcelFile object
    excel_file = writer.book
    #Access the sheet you want to add headings to
    sheet = excel_file['Sheet_1']
    #Put headings into the sheet
    sheet['A1'] = 'Expt 1'  
    sheet['A8'] =  'Expt 2'  
    sheet['A14'] = 'Expt 3'  
    sheet['A20'] = 'Expt 4'  